In [ ]:
ϵ = 1.0
ϵ2 = 100
iteration = 0
ll = 0.0
likelihood = []
alist1 = [a[1]]
alist2 = [a[2]]
alist3 = [a[3]]
dlist1 = [D[1]]
dlist2 = [D[2]]
dlist3 = [D[3]]
# p = Plots.plot(alist1);
# while iteration <= 100
while ϵ2 > 1e-4
    pa = a
    pD = D
    pA = A
    pll = ll
    ll = 0.0
    α = zeros((maxlength, K, tracknum));
    β = zeros((maxlength, K, tracknum));
    γ = zeros((maxlength, K, tracknum));
    ξ = zeros((maxlength, K, K, tracknum));
    c = zeros((maxlength, tracknum));
    L = zeros((maxlength, K, tracknum));
    v = zeros((K));

    likelihood!(dR, D, L, dt, error)
    # L = round.(L, digits=4)
    @inbounds for i in 1:tracknum
        α[1, :, i] = a .* L[1, :, i]
        c[1, i] = sum(α[1, :, i])
        α[1, :, i] /= c[1, i]
        @inbounds for t in 2:count(!iszero, L[:,1,i])
            for s2 in 1:K
                for s1 in 1:K
                    α[t,s1,i] += α[t-1,s1,i] * A[s1,s2]
                end
            end
            for j in 1:K
                α[t,j,i] *= L[t,j,i]
                c[t,i] += α[t,j,i]
            end
            for j in 1:K
                α[t,j,i] /= c[t,i]
            end
        end
        @inbounds for s in 1:K
            β[tracklength[i]+1, s, i] = 1
        end
    #         β[tracklength[i]+1, :, i] .= 1
#         @inbounds for t in count(!iszero, L[:,1,i])-1:-1:1
            @inbounds for t in reverse(1:tracklength[i])
            for s1 in 1:K
                for s2 in 1:K
                    β[t,s1,i] += β[t+1,s2,i] * A[s1,s2] * L[t+1,s2,i]
                end
            end
            for s in 1:K
                β[t,s,i] /= c[t+1,i]
            end
    #             β[t, :, i] = L[t+1, :, i]' .* β[t+1, :, i]' * A'
    #             β[t, :, i] /= c[t+1, i]
        end
        @inbounds for t in 1:count(!iszero, L[:,1,i])
            @inbounds for s in 1:K
                γ[t,s,i] += α[t,s,i] * β[t,s,i]
            end
        end
    #     γ[:, :, i] = α[:,:,i] .* β[:,:,i]
        @inbounds for t in 1:count(!iszero, L[:,1,i])-1
#                 ξ[t, :, :, i] .= (α[t, :, i] * (β[t+1, :, i] .* L[t+1, :, i])') .* A ./ c[t+1, i]
#             end
            @inbounds for s1 in 1:K
                for s2 in 1:K
                    ξ[t, s1, s2, i] = (α[t,s1,i] * A[s1,s2] * L[t+1,s2,i]) / c[t+1, i]
                end
            end
        end
    end
    a = sum(γ[1, :, :], dims=2)[:, 1] ./ sum(γ[1, :, :])
    D = reshape(sum(abs2.(dR) .* γ, dims=(1,3)) ./ (sum(γ, dims=(1,3)) .* (4dt)), K)
#     D .-= abs2(error)/dt
    A = reshape(sum(ξ, dims=(1, 4)), K,K) ./ sum(reshape(sum(ξ, dims=(1, 4)), K,K), dims=2)
    # ϵ = sum(abs.(pa-a)) + sum(abs.(pA-A)) + sum(abs.(pD-D))
    iteration += 1
    replace!(log.(c), -Inf=>0.0)
    ll = sum(c)
#     for i in 1:tracknum
#         ll += log.(c[tracklength[i]+1, i])
#     end
    append!(likelihood, ll)
    append!(dlist1, D[1])
    append!(dlist2, D[2])
    append!(dlist3, D[3])
    append!(alist1, a[1])
    append!(alist2, a[2])
    append!(alist3, a[3])
    ϵ2 = abs(ll-pll)
    if iteration % 25 == 0
        println("iteration = $iteration")
        println("a = $a")
        println("D = $D")
    #         println("parameters = $ϵ")
        println("likelihood = $ϵ2")
#     break
    end
end
println("iteration = $iteration")
println("a = $a")
println("D = $D")
println("parameters = $ϵ")
println("likelihood = $ϵ2")